# Import software libraries

In [59]:
import warnings                                               # Suppress warnings.
warnings.filterwarnings('ignore')

import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.
import re                                                     # Work with regular expressions.

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


import spacy                                                  # Process text.
import nltk                                                   # Process text.
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
nltk.data.path.append('/home/jovyan/work/nltk_data/')


# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]
- pandas 1.1.3
- sqlite3 3.30.1


## Lab 2-1, Reading Data from CSV Files

### Scenario
**Greene City National Bank (GCNB) has given you access to the sources of data that you've agreed
could be useful for achieving its business goals. The data is spread out among different files and
databases, and is stored in different formats.**

In [3]:
complaints_data = pd.read_csv("data/consumer_loan_complaints.csv")
complaints_data.head()

,user_id,Date received,Product,Issue,Consumer complaint narrative,State,ZIP code,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,44fefdad-7045-4be5-890e-12e84ae6fdc9,01/27/2016,Consumer Loan,Account terms and changes,NaN,AL,35180,Phone,01/27/2016,Closed with explanation,Yes,No,1760486
1,c49d5d60-909f-406b-b7ff-51143fcb650b,08/26/2014,Consumer Loan,Account terms and changes,NaN,NC,278XX,Phone,08/29/2014,Closed with non-monetary relief,Yes,No,1001740
2,9b2cd5d2-900e-4052-831f-6489f6d568af,08/22/2012,Consumer Loan,Account terms and changes,NaN,TN,37205,Referral,08/23/2012,Closed with non-monetary relief,Yes,No,140039
3,b7e5b324-268e-4502-81a1-1a025673c2a0,05/07/2013,Consumer Loan,Problems when you are unable to pay,NaN,OH,43081,Web,05/08/2013,Closed with explanation,Yes,Yes,401541
4,684eeb4c-c9c3-4a97-8213-f3962a6c0aba,06/15/2016,Consumer Loan,Managing the line of credit,NaN,NC,27216,Phone,09/08/2016,Closed with non-monetary relief,Yes,No,1970341


#### Each row in the table is a user that submitted a complaint. The columns are as follows:
• user_id: An arbitrary hexadecimal string that uniquely identifies the user.

• Date received: The date the complaint was received by the organization.

• Product: The product the complaint is about.

• Issue: The type of issue the complaint is about.

• Consumer complaint narrative: The text of any written complaints. This field is optional
for the complainant to fill out.

• State: The U.S. state the user resides in.

• ZIP code: The ZIP code the user resides in.

• Submitted via: The method the user took to file the complaint.

• Date sent to company: The date the complaint was sent by the user.

• Company response to consumer: How the organization handled the complaint.

• Timely response?: Whether or not the organization's response was, according to some
metric, given within an acceptable time period.

• Consumer disputed?: Whether or not the user disputed the action the organization took in
response to the complaint.

• Complaint ID: A unique identifier for the complaint itself.

## Lab 2-2, Extracting Data with Database Queries

### Scenario
**The remaining three tables GCNB provided to you are all contained within an SQL database. You
have been given access to this database, but in order to actually retrieve the data, you'll need to
execute some SQL queries. You also want to begin shaping the data so that it's in a more workable
form. For example, the transaction history database currently records each transaction separately,
even transactions made by the same user at different times. Rather than keep it like this, you want to
aggregate the transactions for each user so that it's easier to integrate this data with the other tables
when the time comes.**

In [4]:
conn = sqlite3.connect("data/user_data.db")
conn

In [7]:
query = 'SELECT * FROM users'
users = pd.read_sql(query, conn)
users.head()

,user_id,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,58,management,married,tertiary,no,yes,no,None,261,1,-1,0,None,no,1998-08-23
1,bb92765a-08de-4963-b432-496524b39157,44,technician,single,secondary,no,yes,no,None,151,1,-1,0,None,no,2008-07-15
2,573de577-49ef-42b9-83da-d3cfb817b5c1,33,entrepreneur,married,secondary,no,yes,yes,None,76,1,-1,0,None,no,2002-06-04
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,47,blue-collar,married,None,no,yes,no,None,92,1,-1,0,None,no,1995-06-29
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,33,None,single,None,no,no,no,None,198,1,-1,0,None,no,1995-08-01


**The database file includes a users table, the first few rows of which are printed. Like with the
consumer complaints file, the users are identified using the same hexadecimal ID numbers. You can
consider this the primary key for most of these tables. There are also various columns:**

• age: The age of the user.

• job: The user's job title.

• marital: The user's marital status.

• education: The user's level of education.

• default: Whether the user has defaulted on a loan.

• housing: Whether or not the user has a housing loan.

• loan: Whether or not the user has a personal loan.

• contact: The method the user and organization use to communicate.

• duration: The duration of the last contact session with the user, in seconds.

• campaign: Number of times the user was contacted for the current marketing campaign.

• pdays: Number of days that passed after the user was contacted from a previous campaign.

• previous: Number of times the user was contacted prior to the current campaign.

• poutcome: The result of the previous campaign.

• term_deposit: Whether or not the client subscribed to a term deposit.

• date_joined: The date the user signed up for an account.

In [8]:
users.shape

(45216, 16)

In [9]:
query = 'SELECT * FROM device'
device = pd.read_sql(query, conn)
device.head()

,user_id,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,mobile
1,bb92765a-08de-4963-b432-496524b39157,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,mobile


**The database file also includes a device table. Once again, the user_id field is present. There's
also a device column that lists the user's preferred type of device to use when banking
electronically.**

In [10]:
device.shape

(45117, 2)

In [11]:
query = 'SELECT * FROM transactions'
transactions = pd.read_sql(query, conn)
transactions.head()

,user_id,transaction_id,amount_usd
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,transaction_5180,1332
1,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,transaction_5607,726
2,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,transaction_6765,85
3,573de577-49ef-42b9-83da-d3cfb817b5c1,transaction_6170,1
4,573de577-49ef-42b9-83da-d3cfb817b5c1,transaction_6090,1


**The final table in the database lists transaction information for users. Aside from the familiar
user_id, there are two other columns:**

• transaction_id: A unique identifier for each transaction.

• amount_usd: The total amount of the transaction in U.S. dollars. Positive transactions indicate a
deposit, whereas negative transactions indicate a withdrawal

In [12]:
transactions.shape

(140034, 3)

In [13]:
# Aggregate data on the number of transactions and the total amount.
query = '''
SELECT user_id, COUNT(*) AS number_transactions, SUM(amount_usd) AS total_amount_USD
FROM transactions
GROUP BY user_id
'''
transactions_agg = pd.read_sql(query, conn)
transactions_agg.head()

,user_id,number_transactions,total_amount_USD
0,0001570d-8aed-465e-b547-8981651084ed,3,792
1,000548ed-aa18-4eef-b8ed-68a9126e33ab,2,1044
2,00069959-4d55-460e-bb76-ae13ddbd80a6,5,0
3,000bab00-aec4-4ee2-81a6-1f897c38726b,19,0
4,000cbac8-212f-46fb-b58f-861dada34284,2,399


**Now, you have a table where each user has all of their transactions consolidated into a single row.
The number of those transactions and the total amount of those transactions appear as new
columns.**

In [14]:
transactions_agg.shape

(35211, 3)

## Lab 2-3, Consolidating Data from Multiple Sources

### Scenario
**You've extracted all of the relevant data from each source. The data tables currently live in separate
objects, but they share a common key: user_id. Instead of keeping them all separate, you'll begin
consolidating them using this primary key so they end up as a single table. In particular, you'll create
a master table that includes user demographics, banking information, device usage, and transaction
history. Having this master table will make many forthcoming data science tasks easier.**

In [16]:
# Do a left join, as all users in the users table are of interest.
query = '''
SELECT left_table.*, right_table.device
FROM users AS left_table
LEFT JOIN device AS right_table
ON left_table.user_id = right_table.user_id
'''
users_w_device = pd.read_sql(query, conn)
users_w_device.head()

,user_id,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,58,management,married,tertiary,no,yes,no,None,261,1,-1,0,None,no,1998-08-23,mobile
1,bb92765a-08de-4963-b432-496524b39157,44,technician,single,secondary,no,yes,no,None,151,1,-1,0,None,no,2008-07-15,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,33,entrepreneur,married,secondary,no,yes,yes,None,76,1,-1,0,None,no,2002-06-04,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,47,blue-collar,married,None,no,yes,no,None,92,1,-1,0,None,no,1995-06-29,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,33,None,single,None,no,no,no,None,198,1,-1,0,None,no,1995-08-01,mobile


**You're going to be doing a left join on this data, as all of the users in the users table (left) are of
interest, and you want to merge any records that match the user_id field from the device table
(right).**

In [17]:
users_w_device.shape

(45216, 17)

**There are 45,216 rows and 17 columns in this new table. The number of rows is equal to the original
users table, indicating that the join worked as expected**

In [18]:
conn.close()

**It's always a good idea to close the connection to a database when you're no longer using it. The
next merge you'll perform will use pandas instead of SQL.**

In [20]:
# Merge users_w_device with transactions_agg. Do a right join so users won't be lost.
users_w_devices_and_transactions = transactions_agg.merge(users_w_device, on = "user_id", how = "right")
users_w_devices_and_transactions.head()

,user_id,number_transactions,total_amount_USD,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,3.000,2143.000,58,management,married,tertiary,no,yes,no,None,261,1,-1,0,None,no,1998-08-23,mobile
1,bb92765a-08de-4963-b432-496524b39157,nan,nan,44,technician,single,secondary,no,yes,no,None,151,1,-1,0,None,no,2008-07-15,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,2.000,2.000,33,entrepreneur,married,secondary,no,yes,yes,None,76,1,-1,0,None,no,2002-06-04,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,nan,nan,47,blue-collar,married,None,no,yes,no,None,92,1,-1,0,None,no,1995-06-29,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,1.000,1.000,33,None,single,None,no,no,no,None,198,1,-1,0,None,no,1995-08-01,mobile


**You're creating a new DataFrame that will merge the aggregated transactions data with the
combined users and devices table. You're doing a right join so that users_w_device (right) won't
lose any users that aren't also in transactions_agg (left).**

In [21]:
users_w_devices_and_transactions.shape

(45216, 19)

**As expected, the new table has the same number of rows as the users_w_devices table
(45,216). The table has also grown to include 19 total columns**

## Lab 2-3, Handling Irregular and Unusable Data

### Scenario
**The tables that GCNB sent over have thousands of records, and it's unlikely that they are in a
pristine state. There's a chance they include at least some corrupt or faulty data, whether as a result
of data entry errors or something else. In any case, you need to find any unusable data and deal with
it so that it doesn't cause issues later on. You have a suspicion that some of the user ages may have
been recorded incorrectly, so you'll look for evidence of that and take the appropriate action, if
necessary. There may also be other circumstances that could indicate faulty data, so you'll do some
more investigation.**

In [22]:
# Identify data where age is greater than 150.
users_w_devices_and_transactions[users_w_devices_and_transactions.age > 150]

,user_id,number_transactions,total_amount_USD,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
7228,44fefdad-7045-4be5-890e-12e84ae6fdc9,nan,nan,178,blue-collar,married,primary,no,yes,no,None,691,1,-1,0,None,no,1997-04-28,desktop
10318,9b2cd5d2-900e-4052-831f-6489f6d568af,2.000,3165.000,891,management,married,tertiary,no,yes,no,None,278,2,-1,0,None,no,2003-10-03,mobile


**There are two users whose age is greater than 150, which suggests the data is corrupted. So, you
will remove these rows from the dataset.**

In [24]:
#Drop incorrect data.
users_cleaned = users_w_devices_and_transactions[users_w_devices_and_transactions.age < 150]
users_cleaned.shape

(45214, 19)

**You're creating a new DataFrame with all of the same data, except for the two corrupt rows.
Dropping entire rows with faulty data isn't the only approach you could take, especially if you believe
the values in the other columns are accurate. However, since there are only two affected records out
of tens of thousands, it's safe to just remove them.**

In [25]:
#Identify more potentially erroneous data. Compare age to device.
pd.crosstab(users_cleaned["age"], users_cleaned["device"])

device,desktop,mobile,tablet
age,,,
18,5,6,1
19,10,22,3
20,11,33,6
21,16,44,19
22,30,87,11
...,...,...,...
90,1,1,0
92,1,1,0
93,0,2,0


**The frequency table shows that younger users are much more likely to use electronic devices to do
their banking than people who are 90+ years old. Since the number of devices used by these elderly
users is so sparse (one to two total per age), it may suggest anomalous data. However, this isn't
necessarily the case, so you'll leave the data alone for now. Still, it's worth the effort to at least
investigate potential anomalies.**

## Lab 2-5, Correcting Data Formats

### Scenario
**Thankfully, it seems that most of the data types in the dataset are being cast correctly. However,
you'll need to convert string objects to Boolean values where appropriate, since some columns
should be True or False. Also, dates and times can often cause problems when they're pulled into a
programming environment, especially since they're often just cast as standard strings. While this isn't
necessarily a problem, it's much easier to work with dates and times when they're cast as datetime
objects. So, you'll convert the relevant column from a string object to a datetime.**

In [26]:
users_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45214 entries, 0 to 45215
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              45214 non-null  object 
 1   number_transactions  35215 non-null  float64
 2   total_amount_USD     35215 non-null  float64
 3   age                  45214 non-null  int64  
 4   job                  44926 non-null  object 
 5   marital              45214 non-null  object 
 6   education            43357 non-null  object 
 7   default              45214 non-null  object 
 8   housing              45214 non-null  object 
 9   loan                 45214 non-null  object 
 10  contact              32196 non-null  object 
 11  duration             45214 non-null  int64  
 12  campaign             45214 non-null  int64  
 13  pdays                45214 non-null  int64  
 14  previous             45214 non-null  int64  
 15  poutcome             8255 non-null  

**You can see that date_joined has a data type of object (a string) instead of datetime64 (a
datetime format). A datetime format will make the column easier to work with. Also, none of the
columns are of a Boolean type, and you know that at least some of them should be based on their
"yes" and "no" values.**

**Note: number_transactions should be an integer (a whole number), but
because there are missing values, it defaults to a float (a number with decimal
points).**

In [30]:
users_cleaned.default.value_counts()

no     44398
yes      816
Name: default, dtype: int64

**The default variable has only "yes" or "no" values in string object form. Other variables follow this
pattern, including housing, loan, and term_deposit. It would be better if these were cast as
Booleans.**

In [31]:
#Convert the relevant variables to a Boolean type.
users_cleaned_1 = users_cleaned.copy()   #Work with a new object.
users_cleaned_1.default.map(dict(yes = 1, no = 0)).astype(bool)
users_cleaned_1.default.value_counts()

no     44398
yes      816
Name: default, dtype: int64

In [32]:
# Do the same thing for other Boolean variables.
bool_vars = ["housing", "loan", "term_deposit"]

for var in bool_vars:
    users_cleaned_1[var] = users_cleaned_1[var].map(dict(yes = 1, no = 0)).astype(bool)
    
    print(f"Converted {var} to Boolean.")

Converted housing to Boolean.
Converted loan to Boolean.
Converted term_deposit to Boolean.


In [33]:
users_cleaned_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45214 entries, 0 to 45215
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              45214 non-null  object 
 1   number_transactions  35215 non-null  float64
 2   total_amount_USD     35215 non-null  float64
 3   age                  45214 non-null  int64  
 4   job                  44926 non-null  object 
 5   marital              45214 non-null  object 
 6   education            43357 non-null  object 
 7   default              45214 non-null  object 
 8   housing              45214 non-null  bool   
 9   loan                 45214 non-null  bool   
 10  contact              32196 non-null  object 
 11  duration             45214 non-null  int64  
 12  campaign             45214 non-null  int64  
 13  pdays                45214 non-null  int64  
 14  previous             45214 non-null  int64  
 15  poutcome             8255 non-null  

In [34]:
#Convert date_joined to a datetime format.
users_cleaned_2 = users_cleaned_1.copy()  #Work with a new object.
users_cleaned_2["date_joined"] = pd.to_datetime(users_cleaned_2["date_joined"], format = "%Y-%m-%d")

In [35]:
users_cleaned_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45214 entries, 0 to 45215
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id              45214 non-null  object        
 1   number_transactions  35215 non-null  float64       
 2   total_amount_USD     35215 non-null  float64       
 3   age                  45214 non-null  int64         
 4   job                  44926 non-null  object        
 5   marital              45214 non-null  object        
 6   education            43357 non-null  object        
 7   default              45214 non-null  object        
 8   housing              45214 non-null  bool          
 9   loan                 45214 non-null  bool          
 10  contact              32196 non-null  object        
 11  duration             45214 non-null  int64         
 12  campaign             45214 non-null  int64         
 13  pdays                45214 non-

## Lab 2-6, Deduplicating Data

### Scenario
**Another issue that plagues relatively large datasets is the presence of duplicates, and the GCNB data
is no different. You'll identify any potentially duplicated rows and then remove them from the
dataset. That way, the analysis and modeling you eventually perform won't be skewed by repeated
data.**

In [36]:
#Identify all duplicated data.
duplicated_data = users_cleaned_2[users_cleaned_2.duplicated(keep = False)]
print("Number of rows with duplicated data: ", duplicated_data.shape[0])

Number of rows with duplicated data:  10


In [39]:
duplicated_data

,user_id,number_transactions,total_amount_USD,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
15456,cba59442-af3c-41d7-a39c-0f9bffba0660,2.000,1218.000,57,management,married,tertiary,yes,True,False,cellular,317,6,-1,0,None,False,1993-02-27,desktop
15457,cba59442-af3c-41d7-a39c-0f9bffba0660,2.000,1218.000,57,management,married,tertiary,yes,True,False,cellular,317,6,-1,0,None,False,1993-02-27,desktop
22006,1e826721-b38c-41c2-88f4-4c28b335b1e6,4.000,159.000,31,technician,single,secondary,no,False,False,cellular,129,1,-1,0,None,False,1990-05-10,mobile
22007,1e826721-b38c-41c2-88f4-4c28b335b1e6,4.000,159.000,31,technician,single,secondary,no,False,False,cellular,129,1,-1,0,None,False,1990-05-10,mobile
35415,a2fb8264-d55a-437b-a8e7-9ec4116b76f4,2.000,676.000,34,management,married,tertiary,no,False,False,cellular,156,1,177,1,success,False,1989-09-02,mobile
35416,a2fb8264-d55a-437b-a8e7-9ec4116b76f4,2.000,676.000,34,management,married,tertiary,no,False,False,cellular,156,1,177,1,success,False,1989-09-02,mobile
35623,f49ac08f-b872-4d57-ac82-9b8a9144020d,4.000,117.000,38,blue-collar,married,secondary,no,True,False,cellular,54,1,337,1,failure,False,2010-06-11,mobile
35624,f49ac08f-b872-4d57-ac82-9b8a9144020d,4.000,117.000,38,blue-collar,married,secondary,no,True,False,cellular,54,1,337,1,failure,False,2010-06-11,mobile
36296,ae3b92a2-cad8-434f-8037-9815e2228839,2.000,426.000,43,admin.,single,secondary,no,True,False,cellular,76,2,304,2,failure,False,1992-10-29,desktop
36297,ae3b92a2-cad8-434f-8037-9815e2228839,2.000,426.000,43,admin.,single,secondary,no,True,False,cellular,76,2,304,2,failure,False,1992-10-29,desktop


**This will create a new DataFrame equal to the existing one, except for the identified duplicates.
Also, you'll retrieve a specific user ID that was duplicated before to make sure it only appears once
in the new dataset.**

In [40]:
#Remove the duplicated data
users_cleaned_final = users_cleaned_2[~users_cleaned_2.duplicated()]
users_cleaned_final[users_cleaned_final["user_id"] == "cba59442-af3c-41d7-a39c-0f9bffba0660"]

,user_id,number_transactions,total_amount_USD,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
15456,cba59442-af3c-41d7-a39c-0f9bffba0660,2.000,1218.000,57,management,married,tertiary,yes,True,False,cellular,317,6,-1,0,None,False,1993-02-27,desktop


**This particular user ID only appears once, so it seems the duplicates have been successfully
removed.**

In [41]:
users_cleaned_final.shape

(45209, 19)

**As expected, five rows have been removed from the overall dataset.**

## Lab 2-7, Handling Textual Data.

### Scenario
**The consumer complaints data that you worked with earlier may still be of value to the project,
especially if the team plans to develop natural language processing (NLP) models sometime in the
future. However, handling the textual data inside this file requires a much different approach than
the numeric and categorical data that you've mostly been working with. You need a way to process
the text so that it's more conducive to analysis and machine learning. There are many techniques for
doing so, and you'll apply several of them to the complaint data.**

In [42]:
complaints_data.head()

,user_id,Date received,Product,Issue,Consumer complaint narrative,State,ZIP code,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,44fefdad-7045-4be5-890e-12e84ae6fdc9,01/27/2016,Consumer Loan,Account terms and changes,NaN,AL,35180,Phone,01/27/2016,Closed with explanation,Yes,No,1760486
1,c49d5d60-909f-406b-b7ff-51143fcb650b,08/26/2014,Consumer Loan,Account terms and changes,NaN,NC,278XX,Phone,08/29/2014,Closed with non-monetary relief,Yes,No,1001740
2,9b2cd5d2-900e-4052-831f-6489f6d568af,08/22/2012,Consumer Loan,Account terms and changes,NaN,TN,37205,Referral,08/23/2012,Closed with non-monetary relief,Yes,No,140039
3,b7e5b324-268e-4502-81a1-1a025673c2a0,05/07/2013,Consumer Loan,Problems when you are unable to pay,NaN,OH,43081,Web,05/08/2013,Closed with explanation,Yes,Yes,401541
4,684eeb4c-c9c3-4a97-8213-f3962a6c0aba,06/15/2016,Consumer Loan,Managing the line of credit,NaN,NC,27216,Phone,09/08/2016,Closed with non-monetary relief,Yes,No,1970341


**Recall that each user that filed a complaint is listed in a row, and each column is a different aspect
of the user or their complaint.**

In [43]:
complaints_data.shape

(1824, 13)

In [44]:
complaints_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1824 entries, 0 to 1823
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   user_id                       1824 non-null   object
 1   Date received                 1824 non-null   object
 2   Product                       1824 non-null   object
 3   Issue                         1824 non-null   object
 4   Consumer complaint narrative  44 non-null     object
 5   State                         1801 non-null   object
 6   ZIP code                      1789 non-null   object
 7   Submitted via                 1824 non-null   object
 8   Date sent to company          1824 non-null   object
 9   Company response to consumer  1824 non-null   object
 10  Timely response?              1824 non-null   object
 11  Consumer disputed?            1824 non-null   object
 12  Complaint ID                  1824 non-null   int64 
dtypes: int64(1), objec

**There are only 44 values in the Consumer complaint narrative feature that aren't null. In
other words, most of this data is missing**

In [45]:
complaints_data.Issue.value_counts()

Managing the line of credit            806
Account terms and changes              484
Shopping for a line of credit          301
Problems when you are unable to pay    233
Name: Issue, dtype: int64

**There are several different types of issues, which makes this a good candidate for encoding, so
there's no need to perform any textual processing on this feature.**

In [46]:
complaints_data["Company response to consumer"].value_counts()

Closed with explanation            1291
Closed with non-monetary relief     184
Closed with monetary relief         182
Closed without relief                75
Closed                               65
Closed with relief                   19
Untimely response                     8
Name: Company response to consumer, dtype: int64

**Likewise, this feature can be encoded since it's just categorical in nature. Ultimately, the
Consumer complaint narrative feature is the only one that needs special handling**

In [47]:
#Extract a subset of data to consider only consumer complaints.
print("Number of users with no complaints data: ", complaints_data["Consumer complaint narrative"].isnull().sum())

Number of users with no complaints data:  1780


In [51]:
#Remove records with missing complaint narratives.
text_data = complaints_data[~complaints_data \
                            ["Consumer complaint narrative"].isnull()]\
                            [["user_id", "Consumer complaint narrative"]]
text_data.head()

,user_id,Consumer complaint narrative
53,1a1448a4-bfe5-455f-bc29-dc79ec5fb2c0,"NONE OF YOUR "" MY LOAN IS A '' below apply to ..."
59,5fede48c-096e-4f82-997d-8229007d8318,XX/XX/2014 I received a letter from the IRS st...
65,fd9fc5ff-19bc-424c-880e-c159c110d21f,This was a revolving account in which I paid W...
159,8b220481-5dd3-4ca9-8409-a632987b1072,"I have had a long standing, perfect pay histor..."
232,70fa6ee7-8cfe-4e58-bac0-6fe891620cde,"Loan company applied payments to credit card, ..."


In [52]:
text_data.shape

(44, 2)

In [53]:
sample_text = text_data["Consumer complaint narrative"].iloc[0]
sample_text

'NONE OF YOUR " MY LOAN IS A \'\' below apply to this situation! This was a car loan but the company is providing fraudulent information this is damaging my credit! \n\nRE : MidAtlantic Finance Company Account No. XXXX - NOT TO BE CONFUSED with my current MAF loan MidAtlantic Finance Company has reported several false items to all XXXX credit reporting agencies, and continues to do so. It is damaging my credit so much so that I was told I did n\'t qualify for a mortgage. \n\nMost recently, I settled this account per agreement on XXXX XXXX, XXXX, yet MAF reported it is a payment on the amount claimed owed ( which has been disputed since XXXX XXXX ). But that is just the most recent false information that was reported. It is showing a debt of {$250.00} per month along with XXXX different amounts charged off of the {$950.00} ( plus interest ) and another {$5100.00} that INCLUDES the {$950.00}. Please refer to the following as I NEVER owed MAF {$8100.00} as it reported. That was the origin

**The complaint is full of characters like punctuation, numbers, and common words. You'll need to
streamline this text to make it more amenable to analysis.**

In [61]:
#Tokenize the sample text into sentences.
nlp = spacy.load('/home/jovyan/work/spacy_data/' +
                 'en_core_web_sm/en_core_web_sm-3.0.0/')

document = nlp(sample_text)

In [62]:
for sentence in document.sents:
    print(sentence)

NONE OF YOUR " MY LOAN IS A '' below apply to this situation!
This was a car loan but the company is providing fraudulent information this is damaging my credit!


RE : MidAtlantic Finance Company Account
No.
XXXX - NOT TO BE CONFUSED with my current MAF loan MidAtlantic Finance Company has reported several false items to all XXXX credit reporting agencies, and continues to do so.
It is damaging my credit so much so that I was told I did n't qualify for a mortgage.


Most recently, I settled this account per agreement on XXXX XXXX, XXXX, yet MAF reported it is a payment on the amount claimed owed ( which has been disputed since XXXX XXXX ).
But that is just the most recent false information that was reported.
It is showing a debt of {$250.00} per month along with XXXX different amounts charged off of the {$950.00} ( plus interest ) and another {$5100.00} that INCLUDES the {$950.00}.
Please refer to the following as I NEVER owed MAF {$8100.00} as it reported.
That was the original amoun

**Each sentence identified by spaCy is printed. Note that the line breaks (\n characters in the
document) are being treated as different sentences.**

In [63]:
sentence = nlp('It is showing a debt of {$250.00} per month along ' \
               'with XXXX different amounts charged off of the ' \
               '{$950.00} ( plus interest ) and another {$5100.00} ' \
               'that INCLUDES the {$950.00}.')

In [64]:
for token in sentence:
    print(token.text)

It
is
showing
a
debt
of
{
$
250.00
}
per
month
along
with
XXXX
different
amounts
charged
off
of
the
{
$
950.00
}
(
plus
interest
)
and
another
{
$
5100.00
}
that
INCLUDES
the
{
$
950.00
}
.


In [68]:
#Identify the parts of speech for each token.
pos = []
for token in sentence:
    pos.append({
        "Word": token,
        "Part of speech": token.pos_
    })
pd.DataFrame(pos).head()

,Word,Part of speech
0,It,PRON
1,is,AUX
2,showing,VERB
3,a,DET
4,debt,NOUN


**The table lists what part of speech each token is. For example, It is a pronoun, is is an auxiliary
verb (a verb that affects the tenses and moods of other verbs), { is punctuation, 250.00 is a
number, and so on.**

In [67]:
#Identify stop words.
stop = []

for token in sentence:
    stop.append({
        "Word": token,
        "Stop Word?": token.is_stop
    })
pd.DataFrame(stop).head()

,Word,Stop Word?
0,It,True
1,is,True
2,showing,False
3,a,True
4,debt,False


In [70]:
#Stem the text. Stemming obtains the affix of a word to reduce that word to a base form, making it easier to work with.
text = 'This was a car loan but the company is providing ' \
       'fraudulent information this is damaging my credit!'

print(word_tokenize(text))

['This', 'was', 'a', 'car', 'loan', 'but', 'the', 'company', 'is', 'providing', 'fraudulent', 'information', 'this', 'is', 'damaging', 'my', 'credit', '!']


In [71]:
stemmer = SnowballStemmer(language = "english")

for token in word_tokenize(text):
    print(token, '-->', stemmer.stem(token))

This --> this
was --> was
a --> a
car --> car
loan --> loan
but --> but
the --> the
company --> compani
is --> is
providing --> provid
fraudulent --> fraudul
information --> inform
this --> this
is --> is
damaging --> damag
my --> my
credit --> credit
! --> !


**You can see that some tokens were properly stemmed, but that the resulting stem is not quite a real
word. For example, company became compani, and damaging became damag. That's why a
more advanced technique like lemmatization is usually preferred.**

In [72]:
#Lemmatize the text.
parsed_text = nlp(text)
for token in parsed_text:
    print(token, '-->', token.lemma)

This --> 1995909169258310477
was --> 10382539506755952630
a --> 11901859001352538922
car --> 17545852598994811774
loan --> 8927681708534802266
but --> 14560795576765492085
the --> 7425985699627899538
company --> 6905553075311563409
is --> 10382539506755952630
providing --> 3805418597433244348
fraudulent --> 7365354041363275294
information --> 5552516367584430480
this --> 1995909169258310477
is --> 10382539506755952630
damaging --> 6490178762778191870
my --> 227504873216781231
credit --> 4378518358114130125
! --> 17494803046312582752


**The lemmatization did a better job of obtaining root words. For example, company didn't change
because it's already in the canonical form, and damaging became damage, which is the canonical
form. Also, verbs like was and is became be.**

In [73]:
#Transform the text
def spacy_cleaner(original_text):
    """Cleans text data to be processed.
    Removes punctuation, whitespace, numbers, stopwords from the text
    and lemmatizes each token."""

    final_tokens = []
    parsed_text = nlp(original_text)

    for token in parsed_text:
        if token.is_punct or token.is_space or token.like_num or token.is_stop:
            pass
        else:
            if token.lemma_ == '-PRON-':
                final_tokens.append(str(token))  # Keep pronouns as they are.
            else:
                sc_removed = re.sub('[^a-zA-Z]', '', str(token.lemma_))
                if len(sc_removed) > 1:
                    final_tokens.append(sc_removed)
    joined = ' '.join(final_tokens)
    preprocessed_text = re.sub(r'(.)\1+', r'\1\1', joined)

    return preprocessed_text

Now it's time to actually transform the text data. This function will:

• Use spaCy to tokenize the text.

• Remove punctuation, spaces, numbers, and stop words.

• Obtain the lemma of each word, except for pronouns, which will be kept as is.

• Replace each word with the lemma using a regular expression.

• Remove any other punctuation like parentheses and backslashes using a regular expression.

• Compile the results in a string.

In [74]:
spacy_cleaner(sample_text)

'LOAN apply situation car loan company provide fraudulent information damage credit MidAtlantic Finance Company Account xx confused current MAF loan MidAtlantic Finance Company report false item xx credit reporting agency continue damage credit tell qualify mortgage recently settle account agreement XX XX XX MAF report payment claim owe dispute XX XX recent false information report show debt month xx different amount charge plus interest include refer following owe MAF report original finance XX xx XX XX payment month XX xx xx XX car purchase XX XX finance HOUSE XX XX MAF statement XX XX XX payment MAF XX XX responsible prior late payment MAF record delinquency xx XX credit report know consider own MAF XX XX charge follow reason car total XX XX XX pay xx payment plus additional interest fee XX XX XX insurance company pay XX XX XX leave balance MAF dispute XX XX give payoff XX xx expiration date dispute month give finally MAF send accounting XX xx support claim payment wrongfully charge

In [75]:
# Compare to sample before transformation
sample_text

'NONE OF YOUR " MY LOAN IS A \'\' below apply to this situation! This was a car loan but the company is providing fraudulent information this is damaging my credit! \n\nRE : MidAtlantic Finance Company Account No. XXXX - NOT TO BE CONFUSED with my current MAF loan MidAtlantic Finance Company has reported several false items to all XXXX credit reporting agencies, and continues to do so. It is damaging my credit so much so that I was told I did n\'t qualify for a mortgage. \n\nMost recently, I settled this account per agreement on XXXX XXXX, XXXX, yet MAF reported it is a payment on the amount claimed owed ( which has been disputed since XXXX XXXX ). But that is just the most recent false information that was reported. It is showing a debt of {$250.00} per month along with XXXX different amounts charged off of the {$950.00} ( plus interest ) and another {$5100.00} that INCLUDES the {$950.00}. Please refer to the following as I NEVER owed MAF {$8100.00} as it reported. That was the origin

In [76]:
# Apply transformation to entire dataset.

text_data['consumer_complaints_cleaned'] = \
text_data['Consumer complaint narrative'].apply(lambda x: spacy_cleaner(x))

text_data.head(n = 3)

,user_id,Consumer complaint narrative,consumer_complaints_cleaned
53,1a1448a4-bfe5-455f-bc29-dc79ec5fb2c0,"NONE OF YOUR "" MY LOAN IS A '' below apply to ...",LOAN apply situation car loan company provide ...
59,5fede48c-096e-4f82-997d-8229007d8318,XX/XX/2014 I received a letter from the IRS st...,XX XX receive letter IRS state owe agency ask ...
65,fd9fc5ff-19bc-424c-880e-c159c110d21f,This was a revolving account in which I paid W...,revolving account pay Wells Fargo National Ban...


## Lab 2-8, Loading Data into a Database

### Scenario
**Now that the GCNB data is in a relatively clean state, you want to begin packaging it for the
forthcoming analysis and modeling tasks. There are many formats that you can load this data into;
instead of choosing just one, you'll try out multiple formats to get a feel for how they differ in terms
of storage and integration. First, you'll load the dataset into an SQL database.**

In [77]:
conn = sqlite3.connect("users_data_cleaned.db")

users_cleaned_final.to_sql("users_cleaned_final",
                          conn,
                          if_exists = "replace",
                          index = False)

In [79]:
query = 'SELECT * FROM users_cleaned_final'
pd.read_sql(query, conn).head()

,user_id,number_transactions,total_amount_USD,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,3.000,2143.000,58,management,married,tertiary,no,1,0,None,261,1,-1,0,None,0,1998-08-23 00:00:00,mobile
1,bb92765a-08de-4963-b432-496524b39157,nan,nan,44,technician,single,secondary,no,1,0,None,151,1,-1,0,None,0,2008-07-15 00:00:00,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,2.000,2.000,33,entrepreneur,married,secondary,no,1,1,None,76,1,-1,0,None,0,2002-06-04 00:00:00,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,nan,nan,47,blue-collar,married,None,no,1,0,None,92,1,-1,0,None,0,1995-06-29 00:00:00,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,1.000,1.000,33,None,single,None,no,0,0,None,198,1,-1,0,None,0,1995-08-01 00:00:00,mobile


In [80]:
conn.close()

## Lab 2-9, Loading Data into a DataFrame

### Scenario
**SQL databases are common storage platforms for relational datasets, but they aren't the only option
available to you. In fact, if you plan to do a lot of work in a programming language like Python, you
may want to preserve your data in a DataFrame structure so that it's easier to read from and write
to. This is possible by saving the DataFrame as a binary pickle file, which you'll do in this lab.**

In [81]:
users_cleaned_final.to_pickle("users_cleaned_final.pickle")

In [82]:
pd.read_pickle("users_cleaned_final.pickle").head()

,user_id,number_transactions,total_amount_USD,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,3.000,2143.000,58,management,married,tertiary,no,True,False,None,261,1,-1,0,None,False,1998-08-23,mobile
1,bb92765a-08de-4963-b432-496524b39157,nan,nan,44,technician,single,secondary,no,True,False,None,151,1,-1,0,None,False,2008-07-15,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,2.000,2.000,33,entrepreneur,married,secondary,no,True,True,None,76,1,-1,0,None,False,2002-06-04,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,nan,nan,47,blue-collar,married,None,no,True,False,None,92,1,-1,0,None,False,1995-06-29,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,1.000,1.000,33,None,single,None,no,False,False,None,198,1,-1,0,None,False,1995-08-01,mobile


In [83]:
pd.read_pickle("users_cleaned_final.pickle").info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45209 entries, 0 to 45215
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id              45209 non-null  object        
 1   number_transactions  35210 non-null  float64       
 2   total_amount_USD     35210 non-null  float64       
 3   age                  45209 non-null  int64         
 4   job                  44921 non-null  object        
 5   marital              45209 non-null  object        
 6   education            43352 non-null  object        
 7   default              45209 non-null  object        
 8   housing              45209 non-null  bool          
 9   loan                 45209 non-null  bool          
 10  contact              32191 non-null  object        
 11  duration             45209 non-null  int64         
 12  campaign             45209 non-null  int64         
 13  pdays                45209 non-

## Lab 2-10, Exporting Data to a CSV File.

### Scenario
**Some of the other team members on the project don't use Python and aren't particularly
comfortable with SQL. But they still need a way to work on the same GCNB data that you've been
preparing. So, you'll export the dataset as a comma-separated values (CSV) file, which preserves the
data and its general structure in a text file. That way they can more easily integrate the data into
whatever environments they are using.**

In [84]:
users_cleaned_final.to_csv("users_cleaned_final.csv", index = False)

In [85]:
pd.read_csv("users_cleaned_final.csv").head()

,user_id,number_transactions,total_amount_USD,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,3.000,2143.000,58,management,married,tertiary,no,True,False,NaN,261,1,-1,0,NaN,False,1998-08-23,mobile
1,bb92765a-08de-4963-b432-496524b39157,nan,nan,44,technician,single,secondary,no,True,False,NaN,151,1,-1,0,NaN,False,2008-07-15,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,2.000,2.000,33,entrepreneur,married,secondary,no,True,True,NaN,76,1,-1,0,NaN,False,2002-06-04,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,nan,nan,47,blue-collar,married,NaN,no,True,False,NaN,92,1,-1,0,NaN,False,1995-06-29,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,1.000,1.000,33,NaN,single,NaN,no,False,False,NaN,198,1,-1,0,NaN,False,1995-08-01,mobile


In [87]:
pd.read_csv("users_cleaned_final.csv").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45209 entries, 0 to 45208
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              45209 non-null  object 
 1   number_transactions  35210 non-null  float64
 2   total_amount_USD     35210 non-null  float64
 3   age                  45209 non-null  int64  
 4   job                  44921 non-null  object 
 5   marital              45209 non-null  object 
 6   education            43352 non-null  object 
 7   default              45209 non-null  object 
 8   housing              45209 non-null  bool   
 9   loan                 45209 non-null  bool   
 10  contact              32191 non-null  object 
 11  duration             45209 non-null  int64  
 12  campaign             45209 non-null  int64  
 13  pdays                45209 non-null  int64  
 14  previous             45209 non-null  int64  
 15  poutcome             8252 non-null  